In [ ]:
# Package Imports and set platform
from io import StringIO
import numpy as np
import numpy_indexed as npi
import pandas as pd
import platform
import requests
import scipy as sp
from scipy.sparse import coo_matrix

curr_comp = platform.node()
curr_comp

# Load Data

In [ ]:
# Read in book data
colnames = ['book_id','title','avg_rating','description']

if curr_comp == 'DESKTOP-ARTEMI5' or curr_comp == 'Mandey-Lappy-Toppy':
    orig_url='https://drive.google.com/file/d/15DvRQIdkXVg3qXVkyDm2GsgXVcTmgzYj/view?usp=sharing'
    file_id = orig_url.split('/')[-2]
    dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
    url = requests.get(dwn_url).text
    books_raw = StringIO(url)
    books = pd.read_csv(books_raw,names=colnames)
elif curr_comp == 'sfort-laptop' or curr_comp == 'sfort-desktop':
    books = pd.read_csv('toobig/books.csv',names=colnames)

books

In [ ]:
# Read in interactions data
colnames = ['user_id','book_id','is_read','rating']

if curr_comp == 'DESKTOP-ARTEMI5':
    interactions = pd.read_csv('C:\\DSCI478_Project_Files\\interactions.csv',names=colnames)
elif curr_comp == 'Mandey-Lappy-Toppy':
    interactions = pd.read_csv('C:\\Users\HP\\Dropbox\\000000 CSU\\CSU 2022 Spring\\DSCI 478\\Final Project\\interactions.csv',names=colnames)
elif curr_comp == 'sfort-laptop' or curr_comp == 'sfort-desktop':
    interactions = pd.read_csv('toobig/interactions.csv',names=colnames)

# Isolate interactions with non-zero rating and is_read status
interactions = interactions[interactions['is_read']==True]
interactions = interactions[interactions['rating']!=0]
interactions

In [ ]:
# Remove books with no interactions
unique_books_int = interactions.book_id.unique()
for book in books.book_id:
    if book not in unique_books_int:
        books = books[books.book_id != book]

In [ ]:
# Create CSV from reduced book set
books.to_csv('red_books.csv')

# Sparse Matrix

Coordinate sparse matrix encoding works by storing the row index, column index, and data value for each non-zero entry in the matrix. We need to map each `user_id` to a row in our sparse matrix, and each `book_id` to a column in our sparse matrix. So we create a map from unique `user_id`s to integers, and another map from each `book_id` to integers. Then we apply this map and build a sparse matrix from coordinate encoding. 

In [ ]:
# Map between user_ids and index
unique_users = np.array(interactions.user_id.unique())
user_index = np.array(range(len(unique_users)))
user_map = dict(zip(unique_users,user_index))

# Map between book_ids and index
book_ids = np.array(books.book_id)
book_index = np.array(range(len(book_ids)))
book_map = dict(zip(book_ids,book_index))

# Apply maps to interactions set
row = npi.remap(interactions.user_id, list(user_map.keys()), list(user_map.values()))
col = npi.remap(interactions.book_id, list(book_map.keys()), list(book_map.values()))
dat = np.array(interactions.rating)

In [ ]:
# Create sparse matrix in coordinate format
users_size = unique_users.size
books_size = book_ids.size
users = coo_matrix((dat, (row,col)), shape=(users_size,books_size))

# Creating CSV

We need our recommendation system to have no dependence on the original datasets since they are quite large. Therefore we need to make a smaller dataset which carries over all necessary information while being smaller. We can take the coordinate/data pairs used in constructing our COO matrix, and put them into a CSV. We also need to store `book_id`s in order to recreate the bijection defined earlier purely from the data, but `book_id`s is part of the `books.csv` dataset, which is small enough to be stored.

In [ ]:
# Make dataframe of col, row, dat, unique_users, book_ids
users_sparse_df = pd.DataFrame(list(zip(row,col,dat)),
                              columns=['r_index','c_index','data'])

In [ ]:
# Export to CSV
users_sparse_df.to_csv('users_sparse.csv',index=False)